In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [28]:
#Gathering Data
boston_dataset = load_boston()
data = pd.DataFrame(data= boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

,PRICE
0,3.178054
1,3.072693
2,3.546740
3,3.508556
4,3.589059
...,...
501,3.109061
502,3.025291
503,3.173878
504,3.091042


In [23]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1, 11))
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1, 11) # calcualte the mean value of all eleven features
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [4]:
type(features.mean())

pandas.core.series.Series

In [5]:
type(features.mean().values)

numpy.ndarray

In [29]:
features.mean().values.reshape(1, 11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [30]:
regr = LinearRegression().fit(features, target) # Calcuate all theta value
fitted_vals = regr.predict(features)

# Calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [21]:
def get_log_estimate(nr_rooms,
                     students_per_classroom, 
                     next_to_river=False, 
                     high_confidence=True):  #this  function only used three features for input.(But it is possible to use all 11 feature to predict)

    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0

    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calculate Range
    if high_confidence:     # if high confidence then 2 standard deviation
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:                   # if high confidence then 2 standard deviation
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68

    return log_estimate, upper_bound, lower_bound, interval

In [22]:
get_log_estimate(3, 20, next_to_river=True, high_confidence = False) # high confidence = True then 95% otherwise false then 68%

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [20]:
np.median(boston_dataset.target)

21.2

In [33]:
# Challenge: Write the python code that converts the log price estimat using 1970s prices
#as well as the upper and the lower bounds to todays's prices. Round the vaules to the 
#nearest 100 dollars.

ZILLOW_MEDIAN_PRICE = 583.3   # FROM ZILLOW WEBSITE
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(nr_rooms=9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)

#Convet to today's dollars

dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [12]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
     rm -- number of rooms in the property
     ptratio -- number of students per teacher in the classroom for the school in the area 
     chas -- True if the property is next to the river, False otherwise.
     large_range -- True for a 95% prediction interval, False for a 68% interval
    
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again')
        return 

    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   students_per_classroom=ptratio,
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)

    #Convet to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [13]:
get_dollar_estimate(rm=0, ptratio=30, chas=True)

That is unrealistic. Try again


In [14]:
import boston_valuation as val  # call boston_valuation(which is .py file). we could call boston_valuation.py anywhere
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
